In [319]:
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
from pyarrow import feather
import numpy as np
import re, psycopg2
import pandas as pd
from sqlalchemy import create_engine

In [291]:
# Get the list of donwloaded subtitles (with R)
subs = os.listdir(r"D:\Users\witen\OneDrive\HUB\movie_analysis\subtitles")
subs.pop(subs.index("temp"))
subs[0:5]

['#Lucky Number (2015).srt',
 '#Screamers (2016).srt',
 "'Neath the Arizona Skies (1934).srt",
 "'Pimpernel' Smith (1941).srt",
 "'Round Midnight (1986).srt"]